Annotation notebook

In [ ]:
import os
import polars as pl
import polars_bio as pb
from pathlib import Path
from pycomfort import files

In [2]:
# Configure Polars to show more rows and columns
pl.Config.set_tbl_rows(-1)  # Show all rows (or use a large number like 1000)
pl.Config.set_tbl_cols(-1)  # Show all columns (or use a large number like 50)
pl.Config.set_tbl_width_chars(1000)  # Increase table width to prevent column truncation
pl.Config.set_fmt_str_lengths(1000)  # Show longer string values without truncation

polars.config.Config

In [5]:
from pathlib import Path

genobear_folder = (Path.home() / ".cache" / "genobear").absolute().resolve()
current_folder = Path.cwd().absolute().resolve()
if current_folder.name == "notebooks":
    current_folder = current_folder.parent

print(current_folder, genobear_folder)

/home/antonkulaga/sources/genobear /home/antonkulaga/.cache/genobear


## Load example files

In [10]:
data = current_folder / "data"
test_data = data / "tests"
files.tprint(test_data)

tests
	antku_small.vcf.gz
	antonkulaga.vcf
	antku_small.vcf
	antonkulaga.vcf.gz


In [23]:
import polars_bio as pb
pb.read_vcf(str(test_data / "antku_small.vcf"))

ERROR:polars_bio:SchemaError(DuplicateQualifiedField { qualifier: Bare { table: "antku_small" }, name: "end" }, Some(""))


AttributeError: 'NoneType' object has no attribute 'name'

In [20]:
clinvar_df = pb.read_vcf("/home/antonkulaga/.cache/genobear/clinvar/clinvar.vcf.gz")
clinvar_df.head(10)

0rows [00:00, ?rows/s]

chrom,start,end,id,ref,alt,qual,filter,af_esp,af_exac,af_tgp,alleleid,clndn,clndnincl,clndisdb,clndisdbincl,clnhgvs,clnrevstat,clnsig,clnsigconf,clnsigincl,clnsigscv,clnvc,clnvcso,clnvi,dbvarid,geneinfo,mc,oncdn,oncdnincl,oncdisdb,oncdisdbincl,onc,oncincl,oncrevstat,oncscv,oncconf,origin,rs,scidn,scidnincl,scidisdb,scidisdbincl,scirevstat,sci,sciincl,sciscv
str,u32,u32,str,str,str,f64,str,f32,f32,f32,i32,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str]
"""1""",66926,66927,"""3385321""","""AG""","""A""",null,"""""",null,null,null,3544463,"[""Retinitis_pigmentosa""]",null,"[""Human_Phenotype_Ontology:HP:0000547"", ""MONDO:MONDO:0019200"", … ""Orphanet:791""]",null,"[""NC_000001.11:g.66927del""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV005419006""]","""Deletion""","""SO:0000159""",null,null,"""OR4F5:79501""","[""SO:0001627|intron_variant""]",null,null,null,null,null,null,null,null,null,"[""0""]",null,null,null,null,null,null,null,null,null
"""1""",69134,69134,"""2205837""","""A""","""G""",null,"""""",null,null,null,2193183,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69134A>G""]","[""criteria_provided"", ""_single_submitter""]","[""Likely_benign""]",null,null,"[""SCV003526545""]","""single_nucleotide_variant""","""SO:0001483""","[""ClinGen:CA502008""]",null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]","[""781394307""]",null,null,null,null,null,null,null,null
"""1""",69308,69308,"""3925305""","""A""","""G""",null,"""""",null,null,null,4039319,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69308A>G""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV006120601""]","""single_nucleotide_variant""","""SO:0001483""",null,null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]",null,null,null,null,null,null,null,null,null
"""1""",69314,69314,"""3205580""","""T""","""G""",null,"""""",null,null,null,3374047,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69314T>G""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV004995495""]","""single_nucleotide_variant""","""SO:0001483""","[""ClinGen:CA338197388""]",null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]","[""2521653848""]",null,null,null,null,null,null,null,null
"""1""",69404,69404,"""3925306""","""T""","""C""",null,"""""",null,null,null,4039320,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69404T>C""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV006120602""]","""single_nucleotide_variant""","""SO:0001483""",null,null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]",null,null,null,null,null,null,null,null,null
"""1""",69423,69423,"""3205581""","""G""","""A""",null,"""""",null,null,null,3374048,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69423G>A""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV004995496""]","""single_nucleotide_variant""","""SO:0001483""","[""ClinGen:CA338197763""]",null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]","[""2521654033""]",null,null,null,null,null,null,null,null
"""1""",69581,69581,"""2252161""","""C""","""G""",null,"""""",null,null,null,2238986,"[""not_specified""]",null,"[""MedGen:CN1

In [18]:
pb.read_vcf(str(test_data / "antku_small.vcf"))

ERROR:polars_bio:SchemaError(DuplicateQualifiedField { qualifier: Bare { table: "antku_small" }, name: "end" }, Some(""))


AttributeError: 'NoneType' object has no attribute 'name'

## Clinvar test

In [8]:
clinvar = genobear_folder / "clinvar" / "clinvar.parquet"
clinvar_df = pl.scan_parquet(clinvar)
clinvar_df.head(10).collect()

chrom,start,end,id,ref,alt,qual,filter,af_esp,af_exac,af_tgp,alleleid,clndn,clndnincl,clndisdb,clndisdbincl,clnhgvs,clnrevstat,clnsig,clnsigconf,clnsigincl,clnsigscv,clnvc,clnvcso,clnvi,dbvarid,geneinfo,mc,oncdn,oncdnincl,oncdisdb,oncdisdbincl,onc,oncincl,oncrevstat,oncscv,oncconf,origin,rs,scidn,scidnincl,scidisdb,scidisdbincl,scirevstat,sci,sciincl,sciscv
str,u32,u32,str,str,str,f64,str,f32,f32,f32,i32,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str]
"""1""",66926,66927,"""3385321""","""AG""","""A""",null,"""""",null,null,null,3544463,"[""Retinitis_pigmentosa""]",null,"[""Human_Phenotype_Ontology:HP:0000547"", ""MONDO:MONDO:0019200"", … ""Orphanet:791""]",null,"[""NC_000001.11:g.66927del""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV005419006""]","""Deletion""","""SO:0000159""",null,null,"""OR4F5:79501""","[""SO:0001627|intron_variant""]",null,null,null,null,null,null,null,null,null,"[""0""]",null,null,null,null,null,null,null,null,null
"""1""",69134,69134,"""2205837""","""A""","""G""",null,"""""",null,null,null,2193183,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69134A>G""]","[""criteria_provided"", ""_single_submitter""]","[""Likely_benign""]",null,null,"[""SCV003526545""]","""single_nucleotide_variant""","""SO:0001483""","[""ClinGen:CA502008""]",null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]","[""781394307""]",null,null,null,null,null,null,null,null
"""1""",69308,69308,"""3925305""","""A""","""G""",null,"""""",null,null,null,4039319,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69308A>G""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV006120601""]","""single_nucleotide_variant""","""SO:0001483""",null,null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]",null,null,null,null,null,null,null,null,null
"""1""",69314,69314,"""3205580""","""T""","""G""",null,"""""",null,null,null,3374047,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69314T>G""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV004995495""]","""single_nucleotide_variant""","""SO:0001483""","[""ClinGen:CA338197388""]",null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]","[""2521653848""]",null,null,null,null,null,null,null,null
"""1""",69404,69404,"""3925306""","""T""","""C""",null,"""""",null,null,null,4039320,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69404T>C""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV006120602""]","""single_nucleotide_variant""","""SO:0001483""",null,null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]",null,null,null,null,null,null,null,null,null
"""1""",69423,69423,"""3205581""","""G""","""A""",null,"""""",null,null,null,3374048,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69423G>A""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV004995496""]","""single_nucleotide_variant""","""SO:0001483""","[""ClinGen:CA338197763""]",null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]","[""2521654033""]",null,null,null,null,null,null,null,null
"""1""",69581,69581,"""2252161""","""C""","""G""",null,"""""",null,null,null,2238986,"[""not_specified""]",null,"[""MedGen:CN1